## Before you start

Let's make sure that we have access to GPU. We can use `nvidia-smi` command to do that.

In [1]:
!nvidia-smi

Mon Jun  3 15:17:45 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla P100-PCIE-16GB           Off | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0              27W / 250W |      0MiB / 16384MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
# import os
# HOME = os.getcwd()
# print(HOME)

## Install YOLOv8

In [3]:
# Pip install method (recommended)

!pip install ultralytics==8.0.196

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.196 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla P100-PCIE-16GB, 16276MiB)
Setup complete ✅ (4 CPUs, 31.4 GB RAM, 5643.4/8062.4 GB disk)


In [4]:
from IPython import display
display.clear_output()

from IPython.display import Image
from ultralytics import YOLO

## Preparing a custom dataset

In [5]:
# !mkdir {HOME}/datasets
# %cd {HOME}/datasets

!pip install roboflow --quiet

from roboflow import Roboflow
rf = Roboflow(api_key="AgRivcdffivrgaoMEErV")
project = rf.workspace("driver-wlf6m").project("driving-monitoring-system")
version = project.version(2)
dataset = version.download("yolov8")

Reason for being yanked: deprecated, use 4.8.0.76
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
jupyterlab 4.1.6 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
jupyterlab-lsp 5.1.0 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
libpysal 4.9.2 requires packaging>=22, but you have packaging 21.3 which is incompatible.
libpysal 4.9.2 requires shapely>=2.0.1, but you have shapely 1.8.5.post1 which is incompatible.
momepy 0.7.0 requires shapely>=2, but you have shapely 1.8.5.post1 which is incompatible.
osmnx 1.9.2 requires shapely>=2.0, but you have shapely 1.8.5.post1 which is incompatible.
spopt 0.6.0 requires shapely>=2.0.1, but you have shapely 1.8.5.post1 which is incompatible.
tensorflow 2.15.0 requires keras<


Extracting Dataset Version Zip to driving-monitoring-system-2 in yolov8:: 100%|██████████| 49836/49836 [00:06<00:00, 8168.36it/s]


## Custom training

Don't forget to add `resume` parameter so that you can continue your training

In [6]:
# %cd {HOME}

!yolo task=detect mode=train model=yolov8s.pt data={dataset.location}/data.yaml epochs=100 imgsz=640 plots=True save=True batch=-1

100%|██████████████████████████████████████| 21.5M/21.5M [00:00<00:00, 52.4MB/s]
New https://pypi.org/project/ultralytics/8.2.28 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.196 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla P100-PCIE-16GB, 16276MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/kaggle/working/driving-monitoring-system-2/data.yaml, epochs=100, patience=50, batch=-1, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_l

In [7]:
# %cd {HOME}

# !yolo task=detect mode=train model=/kaggle/working/runs/detect/train/weights/best.pt data={dataset.location}/data.yaml epochs=40 imgsz=640 plots=True save=True batch=-1 

In [8]:
# !yolo task=detect mode=train model=/kaggle/working/runs/detect/train/weights/best.pt data={dataset.location}/data.yaml epochs=40 imgsz=640 plots=True save=True batch=-1 resume=True

In [9]:
# !ls {HOME}/runs/detect/train/

In [10]:
# %cd {HOME}
# Image(filename=f'{HOME}/runs/detect/train/confusion_matrix.png', width=600)

In [11]:
# %cd {HOME}
# Image(filename=f'{HOME}/runs/detect/train/results.png', width=600)

# Validate with Custom Data

In [12]:
# %cd {HOME}
# {HOME}/runs/detect/train/weights/best.pt
# !yolo task=detect mode=val model={HOME}/runs/detect/train/weights/best.pt data={dataset.location}/data.yaml

# Inference with Custom Data

In [13]:
# %cd {HOME}
# !yolo task=detect mode=predict model={HOME}/runs/detect/train/weights/best.pt conf=0.25 source={dataset.location}/test/images save=True

In [14]:
# !yolo task=detect mode=predict model={HOME}/runs/detect/train/weights/best.pt conf=0.25 source='https://github.com/Braincore-C24-MS02/ML-Development/blob/main/dataset/video/driver_face_dataset.mp4'